In [36]:
import os
import pandas as pd
import textdistance
import numpy as np

In [17]:
#get all different characters and locations
FOLDER_PATH = os.path.join('..','output_parsed','ner')
characters = []
locations = []
pronouns = ['his','he','she','her','him', 'hers', 'you', 'i', 'me', 'my', 'your']
for f in os.listdir(FOLDER_PATH):
    file = os.path.join(FOLDER_PATH, f)
    df =  pd.read_csv(file, sep=",",engine="python")
    
    for index, row in df.iterrows():
        if (row['ner'] == "PERSON") and (row['text'] not in characters) and  not (any(p.lower() == row['text'].lower() for p in pronouns)):
            characters.append(row['text'])
        if (row['ner'] == "LOCATION") and (row['text'] not in locations):
            locations.append(row['text'])


In [3]:
#find similar location names
data = []
for i in range(len(locations)):
    for j in range(i+1,len(locations)):
        overlap = textdistance.Overlap(qval=2).normalized_similarity(locations[i],locations[j])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(locations[i],locations[j])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(locations[i],locations[j])
        sim = cosine + ja_wi
        if sim > 1.6:
            data.append([locations[i], locations[j], sim])
            
df_loc = pd.DataFrame(data, columns = ["location1", "location2", "similarity"])

In [5]:
df_loc.to_csv('normLocations.csv')

In [18]:
#find similar character names
data = []
for i in range(len(characters)):
    for j in range(i+1,len(characters)):
        overlap = textdistance.Overlap(qval=2).normalized_similarity(characters[i],characters[j])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(characters[i],characters[j])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(characters[i],characters[j])
        sim = overlap + cosine + ja_wi
        if sim > 2.5:
            data.append([characters[i], characters[j], sim, "No"])
  
df_ch = pd.DataFrame(data, columns = ["character1", "character2", "similarity", "normalized"])

In [19]:
df_ch.to_csv('normCharacters.csv')


In [20]:
data = []
for i in range(len(characters)):
    for j in range(i+1,len(characters)):
        overlap = textdistance.Overlap(qval=2).normalized_similarity(characters[i],characters[j])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(characters[i],characters[j])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(characters[i],characters[j])
        sim = overlap + cosine
        if sim > 1.5:
            data.append([characters[i], characters[j], sim, "No"])
  
df_ch2 = pd.DataFrame(data, columns = ["character1", "character2", "similarity", "normalized"])

In [22]:
df_ch2.to_csv('normCharacters2.csv')

In [47]:
FOLDER_PATH = os.path.join('..','output_parsed','ner')
df = pd.concat((pd.read_csv(file) for file in  (os.path.join(FOLDER_PATH, f) for f in os.listdir(FOLDER_PATH)) ))
df["story"] = f
df

,text,ner,story
0,Sherlock Holmes,PERSON,the_yellow_face.csv
1,Baker Street,LOCATION,the_yellow_face.csv
2,his,PERSON,the_yellow_face.csv
3,Holmes,PERSON,the_yellow_face.csv
4,his,PERSON,the_yellow_face.csv
...,...,...,...
407,his,PERSON,the_yellow_face.csv
408,his,PERSON,the_yellow_face.csv
409,Watson,PERSON,the_yellow_face.csv
410,Norbury,PERSON,the_yellow_face.csv


In [48]:
df_person = df[df["ner"] == "PERSON"]
df_counts = df_person.groupby("text").size().reset_index(name='count').sort_values(['count'], ascending=False)
pronouns = ['his','he','she','her','him', 'hers', 'you', 'i', 'me', 'my', 'your']
df_ordered = df_counts[~df_counts["text"].str.lower().isin(pronouns)]
df_ordered

,text,count
500,Holmes,1940
1499,Watson,1015
980,Mr. Holmes,678
1353,Sherlock Holmes,270
705,Lestrade,225
...,...,...
756,Marvin--he,1
117,Bert Stevens,1
759,Mary Fraser,1
760,Mary Jane,1


In [8]:
all_characters = ["Abernetty", "Abrahams", "Achmet", "Mr. Acton", "Hilda Adair", "Ronald Adair", "Adams", "Irene Adler", "Moore Agar", "Agatha", "Dr. Ainstree", "Dost Akbar", "Prince Albert", "Archie", "Atkinson", "Bardle", "Barton", "Baynes", "Boswell", "Bradstreet", "Sam Brown", "James Calhoun", "Thomas De Quincey", "Cook", "Frédéric Cuvier", "Duchess of Devonshire", "Dundas", "Mrs. Etherege", "Gustave Flaubert", "Forbes", "Fordham", "Forrester", "Mr. Fowler", "Freebody", "Émile Gaboriau", "Gladstone", "Gregory", "Tobias Gregson", "Hafiz", "Mr. Hardy", "John Hare", "Hill", "Mycroft Holmes", "Sherlock Holmes", "John B. Hood", "Ezekiah Hopkins", "Stanley Hopkins", "Horace", "Mrs. Hudson", "Thomas J. Jackson", "John the cabman", "Athelney Jones", "Peter Jones", "Abdullah Khan", "King of Scandinavia", "Lanner", "Monsieur Lecoq", "Robert E. Lee", "Lestrade", "Alec MacDonald", "MacKinnon", "Martin", "Mary", "Mary Jane", "McCauley", "Merivale", "Mr. Merryweather", "Montgomery", "James Moriarty", "William Morris", "Morton", "Godfrey Norton", "Elias Openshaw", "John Openshaw", "Joseph Openshaw", "Paramore", "Patterson", "Grice Patersons", "Prendergast", "Duncan Ross", "William Clark Russell", "George Sand", "Sarasate", "Bartholomew Sholto", "Mahomet Singh", "Jonathan Small", "Joseph Smith", "John Clay", "Colonel Lysander Stark", "Dr. Lysander Starr", "Mary Sutherland", "Mr. Sutherland", "Mrs. Sutherland", "Ned Sutherland", "John Swain", "Trpoff", "Mrs. Turner", "Wilhelm Gottsreich Sigismond von Ormstein", "Clotilde Lothman von Saxe-Meningen", "Wallenstein", "John H. Watson", "Mrs. Watson", "Abel White", "Jabez Wilson", "James Windibank", "Yough"]

In [14]:
data = []
for index, row in df_ordered.iterrows():
    matches = []
    weak_matches = []
    similarities = []
    weak_similarities = []
    for char in all_characters:
        overlap = textdistance.Overlap(qval=2).normalized_similarity(char,row["text"])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(char,row["text"])
        jaccard = textdistance.jaccard.normalized_similarity(char,row["text"])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(char,row["text"])
        sim = cosine + overlap
        if sim > 1.5:
            matches.append(char)
            similarities.append(sim)
        elif ((sim <=1.5) and (sim > 0.5)):
            weak_matches.append(char)
            weak_similarities.append(sim)
            
    if matches:
        matches = ' / '.join(matches)
        weak_matches = []
        weak_similarities = []
    if weak_matches:
        weak_matches = ' / '.join(weak_matches)
    data.append([row["count"], row["text"], matches, similarities, weak_matches, weak_similarities, row["text"]])
  
df_char = pd.DataFrame(data, columns = ["count","dirty_character", "strongly_suggested", "similarity", "weakly suggested", "similarity", "clean_character"])

In [29]:
data = []
for index, row in df_ordered.iterrows():
    matches = []
    similarities = []
    for char in all_characters:
        overlap = textdistance.Overlap(qval=2).normalized_similarity(char,row["text"])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(char,row["text"])
        jaccard = textdistance.jaccard.normalized_similarity(char,row["text"])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(char,row["text"])
        sim = cosine + overlap
        if sim > 1.5:
            matches.append(char)
            similarities.append(sim)            
    if matches:
        matches = ' / '.join(matches)
        data.append([row["count"], row["text"], matches, similarities, np.nan])
    else:
        data.append([row["count"], row["text"], np.nan, np.nan, np.nan])
  
df_characters = pd.DataFrame(data, columns = ["count","dirty_character", "strongly_suggested", "similarity", "clean_character"])

In [30]:
df_characters.to_csv('characters_strong_matches.csv')

In [44]:
df_characters.fillna(value=np.nan, inplace = True)
df_characters.head()

,count,dirty_character,strongly_suggested,similarity,clean_character
0,1940,Holmes,Mycroft Holmes / Sherlock Holmes,"[1.6201736729460423, 1.597614304667197]",NaN
1,1015,Watson,John H. Watson / Mrs. Watson,"[1.6201736729460423, 1.7071067811865475]",NaN
2,678,Mr. Holmes,NaN,NaN,NaN
3,270,Sherlock Holmes,Sherlock Holmes,[2.0],NaN
4,225,Lestrade,Lestrade,[2.0],NaN


In [45]:
df_important = df_characters[df_characters["count"]>5]
df_others = df_characters[df_characters["count"]<=5]
df_others = df_others[~df_others["strongly_suggested"].isna()]
df_important_characters = pd.concat([df_important,df_others])
df_important_characters

,count,dirty_character,strongly_suggested,similarity,clean_character
0,1940,Holmes,Mycroft Holmes / Sherlock Holmes,"[1.6201736729460423, 1.597614304667197]",NaN
1,1015,Watson,John H. Watson / Mrs. Watson,"[1.6201736729460423, 1.7071067811865475]",NaN
2,678,Mr. Holmes,NaN,NaN,NaN
3,270,Sherlock Holmes,Sherlock Holmes,[2.0],NaN
4,225,Lestrade,Lestrade,[2.0],NaN
...,...,...,...,...,...
1515,1,Mary Morstan,Mary,[1.5222329678670934],NaN
1524,1,Mahomet,Mahomet Singh,[1.7071067811865475],NaN
1526,1,Major Freebody,Freebody,[1.7337993857053429],NaN
1540,1,Mary Fraser,Mary,[1.547722557505166],NaN


In [53]:
pairs = []
similarities = []
characters = df_ordered["text"]
for index, row in df_important_characters.iterrows():
    match = []
    similarity = []
    for char in characters:
        if char!=row["dirty_character"]:
            overlap = textdistance.Overlap(qval=2).normalized_similarity(char,row["dirty_character"])
            cosine = textdistance.Cosine(qval=2).normalized_similarity(char,row["dirty_character"])
            sim = cosine + overlap
            if sim > 1.5:
                match.append(char)
                similarity.append(sim)            
    if match:
        match = ' / '.join(match)
        pairs.append(match)
        similarities.append(similarity)
    else:
        pairs.append(np.nan)
        similarities.append(np.nan)
  

In [70]:
df_important_characters["weakly_suggested"] = pairs
df_important_characters["similarities"] = similarities
df_important_characters

,count,dirty_character,strongly_suggested,similarity,clean_character,weakly_suggested,similarities
0,1940,Holmes,Mycroft Holmes / Sherlock Holmes,"[1.6201736729460423, 1.597614304667197]",NaN,Mr. Holmes / Sherlock Holmes / Mr. Sherlock Ho...,"[1.7453559924999298, 1.597614304667197, 1.5270..."
1,1015,Watson,John H. Watson / Mrs. Watson,"[1.6201736729460423, 1.7071067811865475]",NaN,Mrs. Watson / Dr. Watson / John H. Watson,"[1.7071067811865475, 1.7453559924999298, 1.620..."
2,678,Mr. Holmes,NaN,NaN,NaN,Holmes / Mr. Sherlock Holmes / Mr. Busybody Ho...,"[1.7453559924999298, 1.7071067811865475, 1.707..."
3,270,Sherlock Holmes,Sherlock Holmes,[2.0],NaN,Holmes / Mr. Sherlock Holmes / Sherlock / Mist...,"[1.597614304667197, 1.8819171036881968, 1.7071..."
4,225,Lestrade,Lestrade,[2.0],NaN,Mr. Lestrade / G. Lestrade / Messrs. Lestrade,"[1.7977240352174655, 1.8366600265340756, 1.683..."
...,...,...,...,...,...,...,...
1515,1,Mary Morstan,Mary,[1.5222329678670934],NaN,Mary / Morstan / Miss Mary Morstan,"[1.5222329678670934, 1.7385489458759964, 1.829..."
1524,1,Mahomet,Mahomet Singh,[1.7071067811865475],NaN,Mahomet Singh,[1.7071067811865475]
1526,1,Major Freebody,Freebody,[1.7337993857053429],NaN,NaN,NaN
1540,1,Mary Fraser,Mary,[1.547722557505166],NaN,Mary / Fraser / Miss Mary Fraser / Mary--Mary ...,"[1.547722557505166, 1.7071067811865475, 1.8164..."


In [71]:
df_important_characters.to_csv("characterNormalization.csv",index=False)

In [66]:
df_important_characters["weakly_suggested"].isna().sum()


73

In [ ]:
for index, row in df_ordered.iterrows():
    for index2, row2 in df_ordered.iterrows():
        overlap = textdistance.Overlap(qval=2).normalized_similarity(row["text"], row2["text"])
        cosine = textdistance.Cosine(qval=2).normalized_similarity(row["text"], row2["text"])
        ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(row["text"], row2["text"])
        sim = overlap + cosine + ja_wi
        if sim > 2.5:
            data.append([row["count"],row["text"], row2["text"], sim, "No"])
  
df_character_sim = pd.DataFrame(data, columns = ["count","character1", "character2", "similarity", "normalized"])

In [ ]:
df_character_sim

In [181]:
data = []
for index, row in df_ordered.iterrows():
        max_sim = 0
        max_match = ''
        for char in all_characters:
            overlap = textdistance.Overlap(qval=2).normalized_similarity(char,row["text"])
            cosine = textdistance.Cosine(qval=2).normalized_similarity(char,row["text"])
            #ja_wi = textdistance.JaroWinkler(qval=2).normalized_similarity(char,row["text"])
            sim = overlap + cosine
            if sim > max_sim:
                max_sim = sim
                max_match = char
        data.append([row["count"],row["text"],max_match, max_sim])
  
df_character = pd.DataFrame(data, columns = ["count","dirty_character", "clean_character", "similarity"])


In [182]:
df_character.iloc[:300]

,count,dirty_character,clean_character,similarity
0,1940,Holmes,Mycroft Holmes,1.620174
1,1015,Watson,Mrs. Watson,1.707107
2,678,Mr. Holmes,Mycroft Holmes,1.221367
3,270,Sherlock Holmes,Sherlock Holmes,2.000000
4,225,Lestrade,Lestrade,2.000000
...,...,...,...,...
295,5,Brenda,Prendergast,1.024264
296,5,Esher,Mrs. Etherege,0.788675
297,5,Oldacre,Hilda Adair,0.591532
298,5,Mr. Joseph Stangerson,Joseph Smith,1.108304


In [183]:
df_character.to_csv('important_characters.csv')